In [14]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

import pyro
from pyro.infer.mcmc import MCMC
import pyro.distributions as dist

from kernel.sghmc import SGHMC
from kernel.sgld import SGLD
from kernel.sgd import SGD

pyro.set_rng_seed(101)

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return(len(self.data))
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [3]:
BATCH_SIZE = 500
NUM_EPOCHS = 800
WARMUP_EPOCHS = 50

In [4]:
train_dataset = datasets.MNIST('./data', train=True, download=True)

test_dataset = datasets.MNIST('./data', train=False, download=True)

perm = torch.randperm(len(train_dataset))
train_idx = perm[:len(train_dataset)*5//6]
val_idx = perm[len(train_dataset)*5//6:]
    
mean = 0.1307
std = 0.3081

# scale and normalise the datasets
X_train = train_dataset.data[train_idx] / 255.0
Y_train = train_dataset.targets[train_idx]

X_val = train_dataset.data[val_idx] / 255.0 
Y_val = train_dataset.targets[val_idx]

X_test = (test_dataset.data / 255.0 - mean) / std
Y_test = test_dataset.targets

# redefine the datasets
train_dataset = Dataset(X_train, Y_train)
val_dataset = Dataset(X_val, Y_val)
test_dataset = Dataset(X_test, Y_test)

# setup the dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
PyroLinear = pyro.nn.PyroModule[torch.nn.Linear]
    
class BNN(pyro.nn.PyroModule):
    
    def __init__(self, input_size, hidden_size, output_size, prec=1.):
        super().__init__()
        # prec is a kwarg that should only used by SGD to set the regularization strength 
        # recall that a Guassian prior over the weights is equivalent to L2 norm regularization in the non-Bayes setting
        
        # TODO add gamma priors to precision terms
        self.fc1 = PyroLinear(input_size, hidden_size)
        self.fc1.weight = pyro.nn.PyroSample(dist.Normal(0., prec).expand([hidden_size, input_size]).to_event(2))
        self.fc1.bias   = pyro.nn.PyroSample(dist.Normal(0., prec).expand([hidden_size]).to_event(1))
        
        self.fc2 = PyroLinear(hidden_size, output_size)
        self.fc2.weight = pyro.nn.PyroSample(dist.Normal(0., prec).expand([output_size, hidden_size]).to_event(2))
        self.fc2.bias   = pyro.nn.PyroSample(dist.Normal(0., prec).expand([output_size]).to_event(1))
        
        self.relu = torch.nn.ReLU()
        self.log_softmax = torch.nn.LogSoftmax(dim=1)

    def forward(self, x, y=None):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.log_softmax(x)# output (log) softmax probabilities of each class
        
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Categorical(logits=x), obs=y)

In [6]:
LR = 2e-6
MOMENTUM_DECAY = 0.01
NUM_STEPS = 1

bnn = BNN(28*28, 100, 10)

sghmc = SGHMC(bnn,
              subsample_positions=[0, 1],
              batch_size=BATCH_SIZE,
              learning_rate=LR,
              momentum_decay=MOMENTUM_DECAY,
              num_steps=NUM_STEPS)

sghmc_mcmc = MCMC(sghmc, num_samples=len(train_dataset)//BATCH_SIZE, warmup_steps=0)
# full posterior predictive 
full_predictive = torch.FloatTensor(10000, 10)
full_predictive.zero_()

for epoch in range(1+NUM_EPOCHS + WARMUP_EPOCHS):
    sghmc_mcmc.run(X_train, Y_train)
    
    if epoch >= WARMUP_EPOCHS:
        
        sghmc_samples = sghmc_mcmc.get_samples()
        predictive = pyro.infer.Predictive(bnn, posterior_samples=sghmc_samples)
        start = time.time()
        
        with torch.no_grad():
            epoch_predictive = None
            for x, y in val_loader:
                if epoch_predictive is None:
                    epoch_predictive = predictive(x)['obs'].to(torch.int8)
                else:
                    epoch_predictive = torch.cat((epoch_predictive, predictive(x)['obs'].to(torch.int8)), dim=1)
                    
            for sample in epoch_predictive:
                predictive_one_hot = F.one_hot(sample, num_classes=10)
                full_predictive = full_predictive + predictive_one_hot
                
            full_y_hat = torch.argmax(full_predictive, dim=1)
            total = Y_val.shape[0]
            correct = int((full_y_hat == Y_val).sum())
            
        end = time.time()

        print("Epoch [{}/{}] test accuracy: {:.4f} time: {:.2f}".format(epoch-WARMUP_EPOCHS, NUM_EPOCHS, correct/total, end - start))

Sample:  45%|█████████████████████████████▋                                    | 45/100 [00:01, 50.08it/s, lr=2.00e-06]

KeyboardInterrupt: 

In [19]:
import time

LR = 2e-5
NOISE_RATE = 4e-5
NUM_STEPS = 1

bnn = BNN(28*28, 100, 10)

sgld = SGLD(bnn,
            subsample_positions=[0, 1],
            batch_size=BATCH_SIZE,
            learning_rate=LR,
            noise_rate=NOISE_RATE,
            num_steps=NUM_STEPS)

sgld_mcmc = MCMC(sgld, num_samples=len(train_dataset)//BATCH_SIZE, warmup_steps=0)
# full posterior predictive 
full_predictive = torch.FloatTensor(10000, 10)
full_predictive.zero_()

for epoch in range(1+NUM_EPOCHS + WARMUP_EPOCHS):
    sgld_mcmc.run(X_train, Y_train)
    sgld_samples = sgld_mcmc.get_samples()
    
    if epoch >= WARMUP_EPOCHS:
        
        start = time.time()
        sgld_samples = sgld_mcmc.get_samples()
        predictive = pyro.infer.Predictive(bnn, posterior_samples=sgld_samples)
        
        with torch.no_grad():
            epoch_predictive = None
            for x, y in val_loader:
                if epoch_predictive is None:
                    epoch_predictive = predictive(x)['obs'].to(torch.int64)
                else:
                    epoch_predictive = torch.cat((epoch_predictive, predictive(x)['obs'].to(torch.int64)), dim=1)
            
            for sample in epoch_predictive:
                predictive_one_hot = F.one_hot(sample, num_classes=10)
                full_predictive = full_predictive + predictive_one_hot
                
            full_y_hat = torch.argmax(full_predictive, dim=1)
            total = Y_val.shape[0]
            correct = int((full_y_hat == Y_val).sum())
            
        end = time.time()

        print("Epoch [{}/{}] test accuracy: {:.4f} time: {:.2f}".format(epoch-WARMUP_EPOCHS, NUM_EPOCHS, correct/total, end - start))

Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:02, 44.23it/s, lr=2.00e-05]


Epoch [-50/800] test accuracy: 0.5739 time: 4.44


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:02, 48.60it/s, lr=2.00e-05]


Epoch [-49/800] test accuracy: 0.6134 time: 4.51


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01, 51.18it/s, lr=2.00e-05]


...

Epoch [17/800] test accuracy: 0.6539 time: 4.77


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:02, 44.23it/s, lr=2.00e-05]


Epoch [18/800] test accuracy: 0.6551 time: 4.81


Sample:  85%|████████████████████████████████████████████████████████          | 85/100 [00:01, 56.01it/s, lr=2.00e-05]

KeyboardInterrupt: 

In [20]:
LR = 2e-6
WEIGHT_DECAY=0.1
WITH_MOMENTUM=True
MOMENTUM_DECAY=0.75

bnn = BNN(28*28, 100, 10, prec=1.)

sgd = SGD(bnn,
          subsample_positions=[0, 1],
          batch_size=BATCH_SIZE,
          learning_rate=LR,
          weight_decay=WEIGHT_DECAY,
          with_momentum=WITH_MOMENTUM,
          momentum_decay=MOMENTUM_DECAY)

sgd_mcmc = MCMC(sgd, num_samples=len(train_dataset)//BATCH_SIZE, warmup_steps=0)

for epoch in range(1+NUM_EPOCHS):
    sgd_mcmc.run(X_train, Y_train)
        
    if epoch >= 0:
        
        sgd_samples = sgd_mcmc.get_samples()
        point_estimate = {site : sgd_samples[site][-1, :].unsqueeze(0) for site in sgd_samples.keys()}
        predictive = pyro.infer.Predictive(bnn, posterior_samples=point_estimate)
        start = time.time()
        
        with torch.no_grad():
            total = 0
            correct = 0
            for x, y in val_loader:
                batch_predictive = predictive(x)['obs']
                batch_y_hat = batch_predictive.mode(0)[0]
                total += y.shape[0]
                correct += int((batch_y_hat == y).sum())
            
        end = time.time()

        print("Epoch [{}/{}] test accuracy: {:.4f}".format(epoch, NUM_EPOCHS, correct/total))

Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:02, 47.05it/s, lr=2.00e-06]


Epoch [-50/800] test accuracy: 0.5426


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01, 51.34it/s, lr=2.00e-06]


Epoch [-49/800] test accuracy: 0.5365


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01, 51.45it/s, lr=2.00e-06]


Epoch [-48/800] test accuracy: 0.5006


Sample: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01, 51.03it/s, lr=2.00e-06]


Epoch [-47/800] test accuracy: 0.5541


Warmup:   0%|                                                                                    | 0/100 [00:00, ?it/s]

KeyboardInterrupt: 